En este Jupyter Notebook construiremos un modelo que nos permita predecir quién ganará una carrera y en qué posición quedará un piloto

In [85]:
import pandas as pd
import numpy as np
import requests

In [89]:
season = 2023

url = f'http://api.jolpi.ca/ergast/f1/{season}/driverstandings/'

response = requests.get(url)
print(response.status_code)

content = response.json()

200


In [100]:
content['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']

[{'position': '1',
  'positionText': '1',
  'points': '575',
  'wins': '19',
  'Driver': {'driverId': 'max_verstappen',
   'permanentNumber': '33',
   'code': 'VER',
   'url': 'http://en.wikipedia.org/wiki/Max_Verstappen',
   'givenName': 'Max',
   'familyName': 'Verstappen',
   'dateOfBirth': '1997-09-30',
   'nationality': 'Dutch'},
  'Constructors': [{'constructorId': 'red_bull',
    'url': 'http://en.wikipedia.org/wiki/Red_Bull_Racing',
    'name': 'Red Bull',
    'nationality': 'Austrian'}]},
 {'position': '2',
  'positionText': '2',
  'points': '285',
  'wins': '2',
  'Driver': {'driverId': 'perez',
   'permanentNumber': '11',
   'code': 'PER',
   'url': 'http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez',
   'givenName': 'Sergio',
   'familyName': 'Pérez',
   'dateOfBirth': '1990-01-26',
   'nationality': 'Mexican'},
  'Constructors': [{'constructorId': 'red_bull',
    'url': 'http://en.wikipedia.org/wiki/Red_Bull_Racing',
    'name': 'Red Bull',
    'nationality': 'Austrian'}]},
 

---

In [50]:
import fastf1

In [51]:
session = fastf1.get_session(2024, 17, 'R')
session.load(telemetry=False, weather=True)

req         WARNING 	DEFAULT CACHE ENABLED! (2.63 GB) /Users/javi/Library/Caches/fastf1
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '16', '63', '4', '1', '14', '23', '43', '44', '50', '27', '10', '3', '24', '31', '77', '11', '55', '18', '22']


In [56]:
results = session.results

Cogemos los resultados. Tenemos las siguientes columnas

['DriverId', 'TeamId', 'Position', 'ClassifiedPosition', 'GridPosition', 'Time', 'Status', 'Points']

Adicionalmente añadimos:

['Season', 'Round', 'circuit_id']

In [60]:
results = results.loc[:, ['DriverId', 'TeamId', 'Position', 'ClassifiedPosition', 'GridPosition', 'Time', 'Status', 'Points']]

In [ ]:
# Ponemos el ganador a 0
results.iloc[0, results.columns.get_loc('Time')] = pd.Timedelta(0)

In [78]:
results['Time'] = results['Time'].dt.total_seconds()

In [83]:
# Encontrar el tiempo máximo entre los que terminaron
max_finished_time = results.loc[results['Status'] == 'Finished', 'Time'].max()

# Definir un margen adicional (10 segundos, por ejemplo)
margin = 10

# Asignar tiempo a los que no tienen tiempo registrado
results.loc[results['Status'] == '+1 Lap', 'Time'] = max_finished_time + margin
results.loc[results['Status'] == 'Retired', 'Time'] = max_finished_time + 10 * margin  # Mayor margen para Retired

In [84]:
results[['Time', 'Status']]

,Time,Status
81,0.000,Finished
16,10.910,Finished
63,31.328,Finished
4,36.143,Finished
1,77.098,Finished
14,85.468,Finished
23,87.396,Finished
43,89.541,Finished
44,92.401,Finished
50,93.127,Finished
